# Setting up corpora and semantic models

In [1]:
from abslithist.models import *

## Corpora

### EEBO-TCP

In [2]:
import lltk
eebo_tcp = lltk.load('eebo_tcp')
eebo_tcp.metadata

,_llp_,author,corpus,date,extent,fnfn_xml,genre,id,id_eebo_citation,id_oclc,...,id_stc,id_vid,medium,num_words,ocr_accuracy,publisher,pubplace,title,year,_lltk_
0,eebo_tcp|A00001,"A., fl. 1593.",EEBO_TCP,1593],[68] p.,/Volumes/Present/DH/corpora/eebo/_xml_eebo_tcp...,Prose,A00001,9.9851e+07,,...,ESTC S115782,16252,Prose,18636,0.981273,"Imprinted by R. Bourne? for Richard Jones,",[London :,[The passoinate [sic] morrice],1593,eebo_tcp|A00001
1,eebo_tcp|A00002,"Aylett, Robert, 1583-1655?",EEBO_TCP,1625.,"[4], 123, [1] p.",/Volumes/Present/DH/corpora/eebo/_xml_eebo_tcp...,Verse,A00002,9.98506e+07,,...,ESTC S115415,15849,Verse,27674,0.987533,"Printed by William Stansby,",London :,"The brides ornaments viz. fiue meditations, mo...",1625,eebo_tcp|A00002
2,eebo_tcp|A00003,"Ailesbury, Thomas, fl. 1622-1659.",EEBO_TCP,1623.,"[8], 52 p.",/Volumes/Present/DH/corpora/eebo/_xml_eebo_tcp...,Prose,A00003,9.98373e+07,,...,ESTC S101513,1641,Prose,11900,0.909412,Printed by George Eld for Leonard Becket and R...,London :,A sermon preached at Paules-Crosse the second ...,1623,eebo_tcp|A00003
3,eebo_tcp|A00008,"Wood, William, fl. 1623, attributed name.",EEBO_TCP,[1623],"13, [1] p.",/Volumes/Present/DH/corpora/eebo/_xml_eebo_tcp...,Prose,A00008,9.98489e+07,,...,ESTC S113665,14017,Prose,3083,0.980863,"S.n],",[The Netherlands? :,Considerations vpon the treaty of marriage bet...,1623,eebo_tcp|A00008
4,eebo_tcp|A00009,None,EEBO_TCP,1590.,"12, 15-23, [1] p.",/Volumes/Present/DH/corpora/eebo/_xml_eebo_tcp...,Prose,A00009,9.98536e+07,,...,ESTC S118410,19008,Prose,4678,0.991663,"By Thomas Scarlet for Thomas Nelson,",Imprinted at London :,The coppie of a letter sent into England by a ...,1590,eebo_tcp|A00009
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56345,eebo_tcp|B43992,None,EEBO_TCP,1682],23 p.,/Volumes/Present/DH/corpora/eebo/_xml_eebo_tcp...,Prose,B43992,,ocn 690988694,...,Wing S3878F,,Prose,8521,0.963737,"s.n.,",[Dublin? :,"Sir, I have received the third part of No Prot...",1682,eebo_tcp|B43992
56346,eebo_tcp|B43993,England and Wales. Court of Quarter Sessions o...,EEBO_TCP,[1682],4 p.,/Volumes/Present/DH/corpora/eebo/_xml_eebo_tcp...,Prose,B43993,,ocn 625106362,...,Wing S3879,,Prose,1158,0.991364,"Printed for W.C. and sold by W. Davis,",[London] :,"Sir, I presume you began to wonder I did not s...",1682,eebo_tcp|B43993
56347,eebo_tcp|B43994,None,EEBO_TCP,1685],1 sheet ([1] p.),/Volumes/Present/DH/corpora/eebo/_xml_eebo_tcp...,Verse,B43994,,ocn 625106363,...,Wing S3888,,Verse,454,0.993392,"s.n.,",[London :,Sir T.J.'s speech to his wife and children,1685,eebo_tcp|B43994
56348,eebo_tcp|B43997,None,EEBO_TCP,[ca. 1684],1 sheet ([1] p.).,/Volumes/Present/DH/corpora/eebo/_xml_eebo_tcp...,Verse,B43997,,ocn 767620496,...,Wing S3918,,Verse,401,0.992519,"Printed for A.C. in St. Johns-street.,","London, :","The six-penny whore, or The London frollick. B...",1684,eebo_tcp|B43997


## Generate semantic models

### Skipgrams

In [3]:
source(save_skipgrams_corpus)

def save_skipgrams_corpus(C_lltk,period_len=50,min_year=None,max_year=None,num_proc=1):
    C=C_lltk
    oroot = f'data/models/{C.id}'
    df = C.metadata
    df['period']=df['year'].apply(lambda y: f'{y//period_len*period_len}-{y//period_len*period_len+period_len}')
    if min_year: df=df[df.year>=min_year]
    if max_year: df=df[df.year<max_year]
        
    # prepare
    objs = [
        (
            [C.textd[idx].path_txt for idx in perioddf.id],
            os.path.join(oroot,period,'skipgrams.txt.gz')
        )
        for period,perioddf in sorted(df.groupby('period'))
    ]

    # remove already done
    objs = [(paths,ofn) for paths,ofn in objs if not os.path.exists(ofn)]
        
    # exec
    pmap(
        _do_save_skipgrams_corpus,
        objs,
        num_proc=num_proc
    )

In [4]:
# save_skipgrams_corpus(eebo_tcp)

### Modeling

In [5]:
source(gen_model)

def gen_model(
        path_to_skipgram_file,
        skipgram_size=10,
        num_runs=1,
        num_skips_wanted=None,
        num_workers=8,
        min_count=10,
        num_dimensions=100,
        sg=1,
        num_epochs=None,
        labels=[],
        num_proc=1
        ):
    import gensim,logging
    logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

    # Load skipgrams
    # skips = gensim.models.word2vec.LineSentence(path_to_skipgram_file)

    objs=[]
    for run in range(num_runs):
        # Output filename
        odir=os.path.join(os.path.dirname(path_to_skipgram_file),f'run_{str(run+1).zfill(2)}')
        if not os.path.exists(odir): os.makedirs(odir)
        ofnfn=os.path.join(odir,'model.txt.gz')
        ofnfn_vocab=os.path.join(odir,'vocab.txt')

        obj = (
            path_to_skipgram_file,
            ofnfn,
            ofnfn_vocab,
            dict(
                 workers=num_workers,
                 sg=sg,
                 min_count=min_count,
                 size=num_dimensions,
                 window=skipgram_size
            )
        )
        objs.append(obj)
    
    return pmap(_do_gen_model, objs[:1], num_proc=num_proc)

In [6]:
m=gen_model('data/models/eebo_tcp/1500-1550/skipgrams.txt.gz', num_runs=25, num_proc=4)
m

  0%|          | 0/1 [00:00<?, ?it/s]2021-02-15 09:42:42,239 : INFO : collecting all words and their counts
2021-02-15 09:42:42,241 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2021-02-15 09:42:42,331 : INFO : PROGRESS: at sentence #10000, processed 334026 words, keeping 26581 word types
2021-02-15 09:42:42,426 : INFO : PROGRESS: at sentence #20000, processed 680774 words, keeping 39076 word types
2021-02-15 09:42:42,513 : INFO : PROGRESS: at sentence #30000, processed 1023624 words, keeping 51208 word types
2021-02-15 09:42:42,604 : INFO : PROGRESS: at sentence #40000, processed 1400929 words, keeping 67116 word types
2021-02-15 09:42:42,681 : INFO : PROGRESS: at sentence #50000, processed 1695308 words, keeping 74814 word types
2021-02-15 09:42:42,760 : INFO : PROGRESS: at sentence #60000, processed 2002370 words, keeping 85035 word types
2021-02-15 09:42:42,836 : INFO : PROGRESS: at sentence #70000, processed 2278987 words, keeping 94567 word types
2021

[None]